In [1]:
library(org.Hs.eg.db)
library(dplyr)
library(Seurat)
library(enrichplot)
library(clusterProfiler)
library(RColorBrewer)
library(ggplot2)

载入需要的程序包：AnnotationDbi

载入需要的程序包：stats4

载入需要的程序包：BiocGenerics


载入程序包：‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min


载入需要的程序包：Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


载入需要的程序包：IRanges

载入需要的程序包：S4Vectors


载入程序包：‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following obj

In [2]:
seurat<-readRDS('/media/AnalysisDisk1/xuzekai/20250715_BMI/DEG/rds/MAIT_SLC4A10.rds')
gene_list <- rownames(seurat)

In [3]:
plot_GO_heatmap <- function(comparison = "Overweight_vs_Normal", 
                            direction = "up",
                            celltypes = "cMono_CD14",  # 新增参数
                            base_path = "/media/AnalysisDisk1/xuzekai/20250715_BMI/final_result",
                            gene_list) {
  
  # 读取数据
  link <- read.csv(file.path(base_path, "DAR_BMI_CRE_summary_all_celltypes_noScenic.csv"))
  DEG  <- read.csv(file.path(base_path, "DEG_final_sig.csv"))
  link <- link %>% 
    subset(celltype==celltypes )
  # 筛选指定 celltype 中同时是 DAR 和 DEG 的基因
  DEG_subset <- DEG %>% 
    subset(celltype_RNA==celltypes )%>%subset(comparison_RNA==comparison)
  
  if (direction == "up") {
    target_DEG <- DEG_subset %>% subset(log2FC > 0)
  } else if (direction == "down") {
    target_DEG <- DEG_subset %>% subset(log2FC < 0)
  } else {
    stop("direction must be 'up' or 'down'")
  }
  if (nrow(target_DEG) == 0) stop("No ", direction, " DEGs for: ", comparison)
  
  gene <- unique(target_DEG$gene)
  if (length(gene) == 0) stop("No overlapping genes in celltype: ", celltypes)
  
  all_symbols <- keys(org.Hs.eg.db, keytype = "SYMBOL")
  universe <- gene_list[gene_list %in% all_symbols]
  gene_mapped <- gene[gene %in% all_symbols]
  if (length(gene_mapped) == 0) stop("No mappable genes in: ", celltypes)
  
  # GO 富集分析
  res_atac <- enrichGO(
    gene          = gene_mapped,
    universe      = universe,
    OrgDb         = org.Hs.eg.db,
    ont           = "ALL",
    keyType       = "SYMBOL",
    pAdjustMethod = "BH",
    pvalueCutoff  = 0.05,
    minGSSize     = 5
  )
  
  if (is.null(res_atac) || nrow(res_atac@result) == 0) {
    message("No significant GO terms for ", celltypes, " / ", comparison, " / ", direction)
    return(NULL)
  }
  
  # 添加元信息
  res_df <- res_atac@result %>%
    mutate(
      comparison = comparison,
      direction  = direction,
      celltype   = celltypes,
      group      = paste0(direction, "_", gsub("_vs_", "_", comparison), "_", celltypes)
    )
  # foldChange 映射（用于热图颜色）
  foldChange <- setNames(target_DEG$log2FC, target_DEG$gene)

if (direction == "up") {
  pal <- "YlOrRd"      # 暖色：黄→橙→红（适合高值突出）
  dir_flag <- 1        # 正常方向：高值=红色
} else {
  pal <- "Blues"       # 冷色：浅蓝→深蓝（适合低值/负值突出）
  dir_flag <- -1       # 反转方向：更负 = 更深蓝（可选）
}

p <- heatplot(res_atac, showCategory = 10, foldChange = foldChange) +
  ggtitle(paste0(toupper(direction), " - ", comparison, " (", celltypes, ")"))  +
  # 关键：不要用 theme_void()！而是显式开启所有轴元素
  theme(
    # 坐标轴文本（必须！）
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = "black"),
    axis.text.y = element_text(size = 10, color = "black"),
    
    # 显式开启刻度线和轴线
    axis.ticks = element_line(color = "black", size = 0.2),
    axis.line = element_line(color = "black", size = 0.2),
    
    # 移除背景网格（可选）
    panel.grid = element_blank(),
    panel.background = element_rect(fill = "white"),
    
    # 标题居中
    plot.title = element_text(hjust = 0.5, size = 14, face = "bold")
  )+
  scale_fill_distiller(
    type = "seq",              # 注意：YlOrRd 和 Blues 都是 sequential
    palette = pal,
    direction = dir_flag,
    name = "log₂FC"
  )
  # 生成带 celltype 的文件名
  filename <- paste0(
    direction, "_",
    gsub("_vs_", "_", comparison), "_",
    celltypes, "_GO_heatplot.pdf"
  )
#添加是否受DAR调控
    link_subset<-link%>%subset(is_DAR=='True')
    link_subset<-link_subset%>%subset(celltype==celltypes)
    link_gene<-link_subset$gene
    x_labels <- levels(factor(p$data$Gene)) # 这里假设 category 是 x 轴变量名，请根据实际情况替换

     # 创建颜色向量：如果x轴元素在link_gene中则为红色，否则为黑色
    x_colors <- ifelse(x_labels %in% link_gene, "red", "black")
    # 更新主题以应用新的 x 轴文本颜色设置
    p <- p +  theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 10, color = x_colors))
   
  ggsave(filename, plot = p, width = 12, height = 10)
  message("Saved: ", filename)
  
  return(res_df)
}

In [4]:
base_path = "/media/AnalysisDisk1/xuzekai/20250715_BMI/final_result"
DEG  <- read.csv(file.path(base_path, "DEG_final_sig.csv"))

In [5]:
DEG

X,Unnamed..0,gene,score,log2FC,p_val,p_val_adj,comparison_RNA,celltype_RNA,comp,control
<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
0,1,RPS21,4.731425,0.2349743,2.229493e-06,4.049428e-02,Underweight_vs_Normal,AS_DC,Underweight,Normal
1,2,RPS4Y1,-5.301526,-2.2646627,1.148383e-07,4.171617e-03,Underweight_vs_Normal,AS_DC,Underweight,Normal
2,3,XIST,4.735105,1.8027691,2.189417e-06,3.976638e-02,Underweight_vs_Overweight,AS_DC,Underweight,Overweight
3,4,RPS4Y1,-5.802251,-3.1605410,6.543062e-09,2.376833e-04,Underweight_vs_Overweight,AS_DC,Underweight,Overweight
4,5,XIST,13.140088,0.8304357,1.940005e-39,3.523632e-35,Underweight_vs_Normal,Atypical_Bm_ITGAX,Underweight,Normal
5,6,SRGN,7.758375,0.4304166,8.602420e-15,5.208192e-11,Underweight_vs_Normal,Atypical_Bm_ITGAX,Underweight,Normal
6,7,S100A9,7.476009,0.4431304,7.661338e-14,3.478822e-10,Underweight_vs_Normal,Atypical_Bm_ITGAX,Underweight,Normal
7,8,KLF2,7.374747,0.2633974,1.646576e-13,5.981352e-10,Underweight_vs_Normal,Atypical_Bm_ITGAX,Underweight,Normal
8,9,LYZ,6.568199,0.5083789,5.092750e-11,1.233328e-07,Underweight_vs_Normal,Atypical_Bm_ITGAX,Underweight,Normal


In [6]:
comparisons <- c("Overweight_vs_Normal", "Underweight_vs_Normal",'Underweight_vs_Overweight')
directions  <- c("up", "down")
#celltypes   <- c("cMono_CD14","cMono_IFI44L","Mature_NK_dim_FCGR3A","Bn_TCL1A","MAIT_SLC4A10") # 替换为你实际的 celltype
celltypes   <-unique(DEG$celltype_RNA)
all_results <- list()

for (ct in celltypes) {
  for (comp in comparisons) {
    for (dir in directions) {
      tryCatch({
        res <- plot_GO_heatmap(
          comparison = comp,
          direction = dir,
          celltypes = ct,
          gene_list = gene_list
        )
        if (!is.null(res)) {
          all_results[[paste(ct, comp, dir, sep = "_")]] <- res
        }
      }, error = function(e) {
        message("Failed for ", ct, " / ", comp, " / ", dir, ": ", e$message)
      })
    }
  }
}

# 合并并保存
if (length(all_results) > 0) {
  combined_df <- dplyr::bind_rows(all_results)
  write.csv(combined_df, "GO_enrichment_all_celltypes_conditions.csv", row.names = FALSE)
  message("All GO results saved!")
}

❌ Failed for AS_DC / Overweight_vs_Normal / up: No up DEGs for: Overweight_vs_Normal

❌ Failed for AS_DC / Overweight_vs_Normal / down: No down DEGs for: Overweight_vs_Normal

Warning message:
“`aes_()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>.”
Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂FC' in 'mbcsToSbcs': for ₂ (U+2082)”
✅ Saved: up_Underweight_Normal_AS_DC_GO_heatplot.pdf

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Warnin

In [2]:
sessionInfo()

R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 22.04.4 LTS

Matrix products: default
BLAS/LAPACK: /home/xuzekai/mambaforge/envs/r-signac/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=zh_CN.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=zh_CN.UTF-8        LC_COLLATE=zh_CN.UTF-8    
 [5] LC_MONETARY=zh_CN.UTF-8    LC_MESSAGES=zh_CN.UTF-8   
 [7] LC_PAPER=zh_CN.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=zh_CN.UTF-8 LC_IDENTIFICATION=C       

time zone: Asia/Shanghai
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] ggplot2_4.0.0          RColorBrewer_1.1-3     clusterProfiler_4.14.6
 [4] enrichplot_1.26.6      Seurat_5.3.0           SeuratObject_5.2.0    
 [7] sp_2.2-0               dplyr_1.1.4            org.Hs.eg.db_3.20